In [ ]:
import torch
import numpy as np
from dataLoader import yield_test_data
from Model import Net 
import torch.nn.functional as F
from torch.autograd import Variable
from torch_geometric.data import Data
import random
# 引入attribute向量
attribute_embedding = np.load("generateData/Attribute_High_Embedding.npy") 
shop_size = attribute_embedding.shape[0]+1    #pad 0
# 引入category向量
category_embedding = np.load("generateData/Category_Embedding.npy")
# shop特征矩阵
shop_matrix = np.concatenate((attribute_embedding, category_embedding), axis=1)
padLine = np.zeros([1, attribute_embedding.shape[1]+category_embedding.shape[1]])
shop_matrix = np.insert(shop_matrix, 0, values=padLine, axis=0)      # pad 0
shop_size = shop_matrix.shape[0]

struct_matrix = np.load("generateData/Spatial_Graph_200m.npy")
edge_index = torch.tensor(struct_matrix, dtype=torch.long).cuda()
data = Data(x=torch.tensor(shop_matrix,dtype=torch.float).cuda(), edge_index=edge_index)

save_path = "MyNetALL.pkl"
model = Net(shop_matrix)
model.load_state_dict(torch.load(save_path))
model.eval()

if torch.cuda.is_available(): 
    model.cuda()

userSum = 0
testPOISum = 0
recommendPOISum_5 = 0
recommendPOISum_10 = 0
hit_5 = 0
hit_10 = 0
for test_data in yield_test_data(1):
    poiLength = len(test_data["poiIndex"])
    samplePOILength = poiLength+100
    allPOIList = test_data["poiIndex"].copy()
    while len(allPOIList) < samplePOILength:
        num = random.randint(1, shop_size-1)
        if num not in allPOIList:
            allPOIList.append(num)
    userIndex = test_data["userIndex"] * samplePOILength
    userHistoryIndex = [test_data["userHistoryIndex"][0] for i in range(samplePOILength)]
    poiIndex = test_data["poiIndex"]
    labelIndex = test_data["labelIndex"]
    
    user = np.array(userIndex, dtype=np.int32)
    poi = np.array(poiIndex, dtype=np.int32)
    userHistory = np.array(userHistoryIndex, dtype=np.int32)
    allPOI = np.array(allPOIList, dtype=np.int32)   
    userHistoryLength = [len(x) for x in userHistoryIndex]
    
    shop_idxs = Variable(torch.LongTensor(list(range(shop_size))))
    batch_user = Variable(torch.LongTensor(user))
    batch_userHistory = Variable(torch.LongTensor(userHistory))
    batch_userHistoryLength = Variable(torch.LongTensor(userHistoryLength))
    allPOI = Variable(torch.LongTensor(allPOI))

    if torch.cuda.is_available():
        shop_idxs, batch_user, batch_userHistory, batch_userHistoryLength, allPOI = shop_idxs.cuda(), batch_user.cuda(), batch_userHistory.cuda(), batch_userHistoryLength.cuda(), allPOI.cuda()
    model.eval()
    with torch.no_grad():
        outputs = model(shop_idxs, data, batch_user, allPOI, batch_userHistory, batch_userHistoryLength)
        outputs = F.softmax(outputs, dim=1)
        baseScore = torch.LongTensor([1, 2, 3, 4, 5]).cuda()
        scoreList = torch.sum(outputs*baseScore, dim=1)
        userSum += 1
        testPOISum += len(poiIndex)
        # top-5时
        values_5, indices_5 = torch.topk(scoreList, 5)    #k代表指标的top-k
        recommendPOISum_5 += 5
        indices_5 = indices_5.cpu().numpy()           # gpu tensor转cpu numpy
        for poiId in poiIndex:
            if allPOIList.index(poiId) in indices_5:
                hit_5 += 1

        # top-10时
        values_10, indices_10 = torch.topk(scoreList, 10)    #k代表指标的top-k
        recommendPOISum_10 += 10
        indices_10 = indices_10.cpu().numpy()           # gpu tensor转cpu numpy
        for poiId in poiIndex:
            if allPOIList.index(poiId) in indices_10:
                hit_10 += 1
        print("Now, User:%d, Precision-5:%.6f, Recall-5:%.6f, Precision-10:%.6f, Recall-10:%.6f" % (userSum, (hit_5/testPOISum), (hit_5/recommendPOISum_5), (hit_10/testPOISum), (hit_10/recommendPOISum_10)))

Loading test data....
Now, User:1, Precision-5:0.000000, Recall-5:0.000000, Precision-10:0.200000, Recall-10:0.100000
Now, User:2, Precision-5:0.000000, Recall-5:0.000000, Precision-10:0.111111, Recall-10:0.100000
Now, User:3, Precision-5:0.000000, Recall-5:0.000000, Precision-10:0.071429, Recall-10:0.066667
Now, User:4, Precision-5:0.023810, Recall-5:0.050000, Precision-10:0.095238, Recall-10:0.100000
Now, User:5, Precision-5:0.022222, Recall-5:0.040000, Precision-10:0.088889, Recall-10:0.080000
Now, User:6, Precision-5:0.052632, Recall-5:0.100000, Precision-10:0.105263, Recall-10:0.100000
Now, User:7, Precision-5:0.059701, Recall-5:0.114286, Precision-10:0.104478, Recall-10:0.100000
Now, User:8, Precision-5:0.053333, Recall-5:0.100000, Precision-10:0.093333, Recall-10:0.087500
Now, User:9, Precision-5:0.064103, Recall-5:0.111111, Precision-10:0.102564, Recall-10:0.088889
Now, User:10, Precision-5:0.076087, Recall-5:0.140000, Precision-10:0.108696, Recall-10:0.100000
Now, User:11, Pre

Now, User:87, Precision-5:0.061174, Recall-5:0.112644, Precision-10:0.113608, Recall-10:0.104598
Now, User:88, Precision-5:0.060945, Recall-5:0.111364, Precision-10:0.113184, Recall-10:0.103409
Now, User:89, Precision-5:0.062424, Recall-5:0.114607, Precision-10:0.115055, Recall-10:0.105618
Now, User:90, Precision-5:0.063415, Recall-5:0.115556, Precision-10:0.115854, Recall-10:0.105556
Now, User:91, Precision-5:0.063260, Recall-5:0.114286, Precision-10:0.115572, Recall-10:0.104396
Now, User:92, Precision-5:0.062878, Recall-5:0.113043, Precision-10:0.114873, Recall-10:0.103261
Now, User:93, Precision-5:0.063625, Recall-5:0.113978, Precision-10:0.115246, Recall-10:0.103226
Now, User:94, Precision-5:0.064858, Recall-5:0.117021, Precision-10:0.116745, Recall-10:0.105319
Now, User:95, Precision-5:0.066202, Recall-5:0.120000, Precision-10:0.118467, Recall-10:0.107368
Now, User:96, Precision-5:0.067130, Recall-5:0.120833, Precision-10:0.119213, Recall-10:0.107292
Now, User:97, Precision-5:0.06

Now, User:171, Precision-5:0.059067, Recall-5:0.116959, Precision-10:0.106911, Recall-10:0.105848
Now, User:172, Precision-5:0.058962, Recall-5:0.116279, Precision-10:0.107311, Recall-10:0.105814
Now, User:173, Precision-5:0.058754, Recall-5:0.115607, Precision-10:0.106933, Recall-10:0.105202
Now, User:174, Precision-5:0.058651, Recall-5:0.114943, Precision-10:0.106745, Recall-10:0.104598
Now, User:175, Precision-5:0.058582, Recall-5:0.114286, Precision-10:0.107206, Recall-10:0.104571
Now, User:176, Precision-5:0.058480, Recall-5:0.113636, Precision-10:0.107018, Recall-10:0.103977
Now, User:177, Precision-5:0.058926, Recall-5:0.114124, Precision-10:0.107351, Recall-10:0.103955
Now, User:178, Precision-5:0.058382, Recall-5:0.113483, Precision-10:0.108092, Recall-10:0.105056
Now, User:179, Precision-5:0.058314, Recall-5:0.112849, Precision-10:0.107968, Recall-10:0.104469
Now, User:180, Precision-5:0.057979, Recall-5:0.112222, Precision-10:0.107922, Recall-10:0.104444
Now, User:181, Preci

Now, User:255, Precision-5:0.060413, Recall-5:0.119216, Precision-10:0.112480, Recall-10:0.110980
Now, User:256, Precision-5:0.060341, Recall-5:0.118750, Precision-10:0.112743, Recall-10:0.110937
Now, User:257, Precision-5:0.060222, Recall-5:0.118288, Precision-10:0.112520, Recall-10:0.110506
Now, User:258, Precision-5:0.060355, Recall-5:0.118605, Precision-10:0.112426, Recall-10:0.110465
Now, User:259, Precision-5:0.060284, Recall-5:0.118147, Precision-10:0.112293, Recall-10:0.110039
Now, User:260, Precision-5:0.060298, Recall-5:0.118462, Precision-10:0.112373, Recall-10:0.110385
Now, User:261, Precision-5:0.059969, Recall-5:0.118008, Precision-10:0.112150, Recall-10:0.110345
Now, User:262, Precision-5:0.059621, Recall-5:0.117557, Precision-10:0.111498, Recall-10:0.109924
Now, User:263, Precision-5:0.059482, Recall-5:0.117110, Precision-10:0.111626, Recall-10:0.109886
Now, User:264, Precision-5:0.059501, Recall-5:0.117424, Precision-10:0.111708, Recall-10:0.110227
Now, User:265, Preci

Now, User:339, Precision-5:0.057653, Recall-5:0.116224, Precision-10:0.107989, Recall-10:0.108850
Now, User:340, Precision-5:0.057619, Recall-5:0.115882, Precision-10:0.108219, Recall-10:0.108824
Now, User:341, Precision-5:0.057535, Recall-5:0.115543, Precision-10:0.108353, Recall-10:0.108798
Now, User:342, Precision-5:0.057625, Recall-5:0.115789, Precision-10:0.108847, Recall-10:0.109357
Now, User:343, Precision-5:0.057971, Recall-5:0.116618, Precision-10:0.109275, Recall-10:0.109913
Now, User:344, Precision-5:0.057837, Recall-5:0.116279, Precision-10:0.109023, Recall-10:0.109593
Now, User:345, Precision-5:0.057875, Recall-5:0.116522, Precision-10:0.108840, Recall-10:0.109565
Now, User:346, Precision-5:0.057676, Recall-5:0.116185, Precision-10:0.108465, Recall-10:0.109249
Now, User:347, Precision-5:0.057896, Recall-5:0.116427, Precision-10:0.108627, Recall-10:0.109222
Now, User:348, Precision-5:0.057764, Recall-5:0.116092, Precision-10:0.108665, Recall-10:0.109195
Now, User:349, Preci

...

Now, User:22624, Precision-5:0.054219, Recall-5:0.097339, Precision-10:0.110023, Recall-10:0.098762
Now, User:22625, Precision-5:0.054218, Recall-5:0.097335, Precision-10:0.110022, Recall-10:0.098758
Now, User:22626, Precision-5:0.054225, Recall-5:0.097348, Precision-10:0.110031, Recall-10:0.098767
Now, User:22627, Precision-5:0.054227, Recall-5:0.097353, Precision-10:0.110030, Recall-10:0.098767
Now, User:22628, Precision-5:0.054223, Recall-5:0.097348, Precision-10:0.110021, Recall-10:0.098763
Now, User:22629, Precision-5:0.054224, Recall-5:0.097353, Precision-10:0.110019, Recall-10:0.098763
Now, User:22630, Precision-5:0.054227, Recall-5:0.097357, Precision-10:0.110020, Recall-10:0.098763
Now, User:22631, Precision-5:0.054225, Recall-5:0.097353, Precision-10:0.110015, Recall-10:0.098758
Now, User:22632, Precision-5:0.054224, Recall-5:0.097349, Precision-10:0.110013, Recall-10:0.098754
Now, User:22633, Precision-5:0.054227, Recall-5:0.097353, Precision-10:0.110019, Recall-10:0.098758


Now, User:22706, Precision-5:0.054226, Recall-5:0.097305, Precision-10:0.110057, Recall-10:0.098745
Now, User:22707, Precision-5:0.054227, Recall-5:0.097309, Precision-10:0.110064, Recall-10:0.098754
Now, User:22708, Precision-5:0.054226, Recall-5:0.097305, Precision-10:0.110062, Recall-10:0.098749
Now, User:22709, Precision-5:0.054229, Recall-5:0.097309, Precision-10:0.110068, Recall-10:0.098754
Now, User:22710, Precision-5:0.054233, Recall-5:0.097314, Precision-10:0.110071, Recall-10:0.098754
Now, User:22711, Precision-5:0.054232, Recall-5:0.097310, Precision-10:0.110073, Recall-10:0.098754
Now, User:22712, Precision-5:0.054235, Recall-5:0.097314, Precision-10:0.110079, Recall-10:0.098758
Now, User:22713, Precision-5:0.054238, Recall-5:0.097319, Precision-10:0.110090, Recall-10:0.098767
Now, User:22714, Precision-5:0.054237, Recall-5:0.097314, Precision-10:0.110089, Recall-10:0.098763
Now, User:22715, Precision-5:0.054241, Recall-5:0.097319, Precision-10:0.110093, Recall-10:0.098763


Now, User:22788, Precision-5:0.054197, Recall-5:0.097192, Precision-10:0.110064, Recall-10:0.098688
Now, User:22789, Precision-5:0.054203, Recall-5:0.097205, Precision-10:0.110065, Recall-10:0.098692
Now, User:22790, Precision-5:0.054202, Recall-5:0.097201, Precision-10:0.110064, Recall-10:0.098688
Now, User:22791, Precision-5:0.054200, Recall-5:0.097196, Precision-10:0.110059, Recall-10:0.098684
Now, User:22792, Precision-5:0.054212, Recall-5:0.097218, Precision-10:0.110069, Recall-10:0.098693
Now, User:22793, Precision-5:0.054212, Recall-5:0.097214, Precision-10:0.110068, Recall-10:0.098688
Now, User:22794, Precision-5:0.054215, Recall-5:0.097219, Precision-10:0.110075, Recall-10:0.098693
Now, User:22795, Precision-5:0.054217, Recall-5:0.097223, Precision-10:0.110074, Recall-10:0.098693
Now, User:22796, Precision-5:0.054224, Recall-5:0.097236, Precision-10:0.110077, Recall-10:0.098697
Now, User:22797, Precision-5:0.054223, Recall-5:0.097232, Precision-10:0.110074, Recall-10:0.098693


Now, User:22870, Precision-5:0.054225, Recall-5:0.097167, Precision-10:0.110091, Recall-10:0.098636
Now, User:22871, Precision-5:0.054223, Recall-5:0.097162, Precision-10:0.110085, Recall-10:0.098631
Now, User:22872, Precision-5:0.054222, Recall-5:0.097158, Precision-10:0.110083, Recall-10:0.098627
Now, User:22873, Precision-5:0.054221, Recall-5:0.097154, Precision-10:0.110081, Recall-10:0.098623
Now, User:22874, Precision-5:0.054220, Recall-5:0.097150, Precision-10:0.110080, Recall-10:0.098619
Now, User:22875, Precision-5:0.054217, Recall-5:0.097145, Precision-10:0.110079, Recall-10:0.098619
Now, User:22876, Precision-5:0.054219, Recall-5:0.097150, Precision-10:0.110078, Recall-10:0.098619
Now, User:22877, Precision-5:0.054223, Recall-5:0.097154, Precision-10:0.110081, Recall-10:0.098619
Now, User:22878, Precision-5:0.054221, Recall-5:0.097150, Precision-10:0.110082, Recall-10:0.098619
Now, User:22879, Precision-5:0.054228, Recall-5:0.097163, Precision-10:0.110085, Recall-10:0.098623


Now, User:22952, Precision-5:0.054217, Recall-5:0.097090, Precision-10:0.110093, Recall-10:0.098575
Now, User:22953, Precision-5:0.054213, Recall-5:0.097085, Precision-10:0.110090, Recall-10:0.098575
Now, User:22954, Precision-5:0.054211, Recall-5:0.097081, Precision-10:0.110086, Recall-10:0.098571
Now, User:22955, Precision-5:0.054207, Recall-5:0.097077, Precision-10:0.110078, Recall-10:0.098567
Now, User:22956, Precision-5:0.054206, Recall-5:0.097073, Precision-10:0.110076, Recall-10:0.098562
Now, User:22957, Precision-5:0.054204, Recall-5:0.097068, Precision-10:0.110071, Recall-10:0.098558
Now, User:22958, Precision-5:0.054200, Recall-5:0.097064, Precision-10:0.110068, Recall-10:0.098558
Now, User:22959, Precision-5:0.054207, Recall-5:0.097077, Precision-10:0.110077, Recall-10:0.098567
Now, User:22960, Precision-5:0.054205, Recall-5:0.097073, Precision-10:0.110073, Recall-10:0.098563
Now, User:22961, Precision-5:0.054202, Recall-5:0.097069, Precision-10:0.110067, Recall-10:0.098558
